In [118]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import random
from sklearn import model_selection, preprocessing, svm, metrics
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


%matplotlib inline

In [119]:
class Open_csv:
    
    def __init__(self):
        pass
    
    def open(self,path_to_file):
        data = np.asarray(pd.read_csv(path_to_file, delimiter=",").values)
        return data

In [46]:
def f_1(vector):
    return (vector[2]-vector[3])/(vector[1]+vector[4]) # (high-low)/(open-close)

In [120]:
# garman-klass
def f(vector):
    # vector[1] = open ; vector[2] = high ; vector[3] = low ; vector[4] = close
    return 12*(np.log(vector[2]/vector[3])**2)-(2*np.log(2)-1)*(np.log(vector[4]/vector[1])**2)

In [136]:
# parkinson
def f(vector):
    # vector[1] = open ; vector[2] = high ; vector[3] = low ; vector[4] = close
    return np.log(vector[2]/vector[3])

In [137]:
class Preprocess_data:
    
    def __init__(self):
        self.function = f_1
        self.open_csv = Open_csv()
        self.data = None
        self.returns = []
        self.log_returns = True
        self.depth = 5
        self.X_ = None
        self.X_all = None
        self.y_ = None
        self.y_all = None     
        
    def get_data_stock(self,stock_name):
        self.data = self.open_csv.open("data/"+stock_name+".csv")
    
    def get_stock_return(self):
        for i in range(len(self.data)):
            self.returns.append(self.function(self.data[i]))
        self.returns = np.asarray(self.returns)
        
    def get_log_returns(self):
        if self.log_returns:
            temp = np.empty(len(np.where(self.returns!=0)[0]))
            for i in range(len(np.where(self.returns!=0)[0])):
                temp[i] = self.returns[np.where(self.returns!=0)[0][i]]
            #self.returns = np.log(temp)
            self.returns = (np.log(temp)-np.mean(np.log(temp)))/np.std(np.log(temp))
            
    def compute_X_and_y(self):
        self.X_ = []
        self.y_ = []
        for i in range(len(self.returns)-self.depth):
            temp = self.returns
            self.y_.append(temp[i])
            self.X_.append(np.array(temp[i+1:i+1+self.depth]))
        self.y_ = np.array(self.y_)        
        temp2 = np.empty((len(self.X_)-1,len(self.X_[0])))
        for i in range(len(temp2)):
            temp2[i] = np.array(self.X_[i])
        self.X_ = temp2
        self.y_ = self.y_[:len(self.X_)]
        
    def focus_on_recent_data(self):
        n_data_for_training = 500
        start_index = 0
        self.X_, self.X_all = self.X_[start_index:start_index+n_data_for_training], self.X_
        self.y_, self.y_all = self.y_[start_index:start_index+n_data_for_training], self.y_
    
    def run(self,stock_name,focus_on_recent_data):
        self.get_data_stock(stock_name)
        self.get_stock_return()
        self.get_log_returns()
        self.compute_X_and_y()
        if focus_on_recent_data:
            self.focus_on_recent_data()
            
    def reset(self):
        self.function = f_1
        self.open_csv = Open_csv()
        self.data = None
        self.returns = []
        self.log_returns = True
        self.depth = 5
        self.X_ = None
        self.X_all = None
        self.y_ = None
        self.y_all = None 

In [138]:
class Estimate:
    
    def __init__(self):
        self.X_ = None
        self.y_ = None
        self.optimize_rf = True
        self.optimize_svm = True
        self.ypred_randomforest = None
        self.ypred_svm = None
    
    def cross_validate(self, design_matrix, labels, regressor, cv_folds):
        pred = np.zeros(labels.shape) # Hold all predictions, in correct order.
        for tr, te in cv_folds:        
            scaler = preprocessing.StandardScaler()
            Xtr = scaler.fit_transform(design_matrix[tr,:])
            ytr = labels[tr]
            Xte = scaler.transform(design_matrix[te,:])
            regressor.fit(Xtr, ytr)
            pred[te] = (regressor.predict(Xte))
        return pred

    def random_forest_estimate(self):
        # Set up a stratified 10-fold cross-validation
        skf = model_selection.KFold(n_splits=10)
        skf.get_n_splits(self.X_, self.y_)
        folds = [(tr,te) for (tr,te) in skf.split(self.X_, self.y_)]
        
        if self.optimize_rf:
            param_grid = {'n_estimators':[1,10],
                    'max_features':[1,4],
                    'max_depth':[1,9]}

            clf_rf = GridSearchCV(RandomForestRegressor(),
                           param_grid)

            self.ypred_randomforest = self.cross_validate(self.X_, self.y_, clf_rf, folds)

        else:
            self.ypred_randomforest = self.cross_validate(self.X_,self.y_,RandomForestRegressor(),folds)
    
    def svm_estimate(self):
        # Set up a stratified 10-fold cross-validation
        skf = model_selection.KFold(n_splits=10)
        skf.get_n_splits(self.X_, self.y_)
        folds = [(tr,te) for (tr,te) in skf.split(self.X_, self.y_)]
        if self.optimize_svm:
            param_grid = {'C':[0.01,10]}

            clf_svm = GridSearchCV(svm.SVR(kernel='linear'),
                           param_grid)

            self.ypred_svm = self.cross_validate(self.X_, self.y_, clf_svm, folds)

        else:
            self.ypred_svm = self.cross_validate(self.X_, self.y_, svm.SVR(kernel='linear'), folds)
            
    def run(self,X_,y_):
        self.X_ = X_
        self.y_ = y_
        self.random_forest_estimate()
        self.svm_estimate()
        
    def reset(self):
        self.X_ = None
        self.y_ = None
        self.optimize_rf = True
        self.optimize_svm = True
        self.ypred_randomforest = None
        self.ypred_svm = None

In [139]:
stock_list2 = ['OR.PA', 'ATO.PA', 'SGO.PA', 'EN.PA', 'SU.PA', 'FR.PA', 'ORA.PA', 'UG.PA', 'ACA.PA', 
               'BNP.PA', 'GLE.PA', 'AC.PA', 'LHN.PA', 'FP.PA']

stock_list3 = ["FP.PA"]

stock_list4 = ['AC.PA', 'LHN.PA', 'FP.PA']

stock_list5 = ['BNP.PA', 'GLE.PA', 'AC.PA', 'LHN.PA', 'FP.PA']

In [140]:
preprocess_data = Preprocess_data()
preprocess_data.run(stock_list3[0],focus_on_recent_data=True)
X_ = preprocess_data.X_
y_ = preprocess_data.y_
preprocess_data.reset()

for i in range(1,len(stock_list3)):
    preprocess_data.run(stock_list3[i],focus_on_recent_data=True)
    X_ = np.concatenate((X_,preprocess_data.X_),axis=0)
    y_ = np.concatenate((y_,preprocess_data.y_),axis=0)    
    preprocess_data.reset()


In [141]:
estimate = Estimate()
estimate.run(X_,y_)

In [142]:
ypred_randomforest = estimate.ypred_randomforest
ypred_svm = estimate.ypred_svm
y_ = estimate.y_

In [143]:
# comparaison avec différents prédicteurs "stupides"
y1_ = np.array([np.mean(y_)]*len(y_))
print("dummy estimator : mean of |y1_ - y_|   :   "+str(np.mean(np.abs(y1_-y_))))
print("dummy estimator : explained variance : "+str(metrics.explained_variance_score(y_,y1_)))
print("dummy estimator : mean absolute error : "+str(metrics.mean_absolute_error(y_,y1_)))
print("dummy estimator : mean squared error : "+str(metrics.mean_squared_error(y_,y1_)))
print("dummy estimator : r2 score : "+str(metrics.r2_score(y_,y1_)))
    
# random forest
print("randomforest : mean of |ypred_randomforest - y_|   :   "+str(np.mean(np.abs(ypred_randomforest-y_))))
print("randomforest : explained variance : "+str(metrics.explained_variance_score(y_,ypred_randomforest)))
print("randomforest : mean absolute error : "+str(metrics.mean_absolute_error(y_,ypred_randomforest)))
print("randomforest : mean squared error : "+str(metrics.mean_squared_error(y_,ypred_randomforest)))
print("randomforest : r2 score : "+str(metrics.r2_score(y_,ypred_randomforest)))
    
# svm
print("svm : mean of |ypred_svm - y_|   :   "+str(np.mean(np.abs(ypred_svm-y_))))
print("svm : explained variance : "+str(metrics.explained_variance_score(y_,ypred_svm)))
print("svm : mean absolute error : "+str(metrics.mean_absolute_error(y_,ypred_svm)))
print("svm : mean squared error : "+str(metrics.mean_squared_error(y_,ypred_svm)))
print("svm : r2 score : "+str(metrics.r2_score(y_,ypred_svm)))

dummy estimator : mean of |y1_ - y_|   :   0.6828702946339802
dummy estimator : explained variance : 0.0
dummy estimator : mean absolute error : 0.6828702946339802
dummy estimator : mean squared error : 0.7129551184039646
dummy estimator : r2 score : 0.0
randomforest : mean of |ypred_randomforest - y_|   :   0.6432419121394937
randomforest : explained variance : 0.07542316954166106
randomforest : mean absolute error : 0.6432419121394937
randomforest : mean squared error : 0.6606497784576731
randomforest : r2 score : 0.07336414115853918
svm : mean of |ypred_svm - y_|   :   0.6032010126864337
svm : explained variance : 0.18657150576733572
svm : mean absolute error : 0.6032010126864337
svm : mean squared error : 0.5799738676274501
svm : r2 score : 0.18652120918103376


In [144]:
preprocess_data = Preprocess_data()
preprocess_data.run(stock_list4[0],focus_on_recent_data=True)
X_ = preprocess_data.X_
y_ = preprocess_data.y_
preprocess_data.reset()

for i in range(1,len(stock_list4)):
    preprocess_data.run(stock_list4[i],focus_on_recent_data=True)
    X_ = np.concatenate((X_,preprocess_data.X_),axis=0)
    y_ = np.concatenate((y_,preprocess_data.y_),axis=0)    
    preprocess_data.reset()


In [145]:
estimate = Estimate()
estimate.run(X_,y_)

In [146]:
ypred_randomforest = estimate.ypred_randomforest
ypred_svm = estimate.ypred_svm
y_ = estimate.y_

In [147]:
# comparaison avec différents prédicteurs "stupides"
y1_ = np.array([np.mean(y_)]*len(y_))
print("dummy estimator : mean of |y1_ - y_|   :   "+str(np.mean(np.abs(y1_-y_))))
print("dummy estimator : explained variance : "+str(metrics.explained_variance_score(y_,y1_)))
print("dummy estimator : mean absolute error : "+str(metrics.mean_absolute_error(y_,y1_)))
print("dummy estimator : mean squared error : "+str(metrics.mean_squared_error(y_,y1_)))
print("dummy estimator : r2 score : "+str(metrics.r2_score(y_,y1_)))
    
# random forest
print("randomforest : mean of |ypred_randomforest - y_|   :   "+str(np.mean(np.abs(ypred_randomforest-y_))))
print("randomforest : explained variance : "+str(metrics.explained_variance_score(y_,ypred_randomforest)))
print("randomforest : mean absolute error : "+str(metrics.mean_absolute_error(y_,ypred_randomforest)))
print("randomforest : mean squared error : "+str(metrics.mean_squared_error(y_,ypred_randomforest)))
print("randomforest : r2 score : "+str(metrics.r2_score(y_,ypred_randomforest)))
    
# svm
print("svm : mean of |ypred_svm - y_|   :   "+str(np.mean(np.abs(ypred_svm-y_))))
print("svm : explained variance : "+str(metrics.explained_variance_score(y_,ypred_svm)))
print("svm : mean absolute error : "+str(metrics.mean_absolute_error(y_,ypred_svm)))
print("svm : mean squared error : "+str(metrics.mean_squared_error(y_,ypred_svm)))
print("svm : r2 score : "+str(metrics.r2_score(y_,ypred_svm)))

dummy estimator : mean of |y1_ - y_|   :   0.7278505673984774
dummy estimator : explained variance : 0.0
dummy estimator : mean absolute error : 0.7278505673984774
dummy estimator : mean squared error : 0.844471854864409
dummy estimator : r2 score : 0.0
randomforest : mean of |ypred_randomforest - y_|   :   0.6549906266634918
randomforest : explained variance : 0.1627450621932338
randomforest : mean absolute error : 0.6549906266634918
randomforest : mean squared error : 0.7071718408068222
randomforest : r2 score : 0.16258684438883064
svm : mean of |ypred_svm - y_|   :   0.6424221034989139
svm : explained variance : 0.20794241075677944
svm : mean absolute error : 0.6424221034989139
svm : mean squared error : 0.6703812898588623
svm : r2 score : 0.20615318793957826


In [148]:
preprocess_data = Preprocess_data()
preprocess_data.run(stock_list5[0],focus_on_recent_data=True)
X_ = preprocess_data.X_
y_ = preprocess_data.y_
preprocess_data.reset()

for i in range(1,len(stock_list5)):
    preprocess_data.run(stock_list5[i],focus_on_recent_data=True)
    X_ = np.concatenate((X_,preprocess_data.X_),axis=0)
    y_ = np.concatenate((y_,preprocess_data.y_),axis=0)    
    preprocess_data.reset()


In [149]:
estimate = Estimate()
estimate.run(X_,y_)

In [150]:
ypred_randomforest = estimate.ypred_randomforest
ypred_svm = estimate.ypred_svm
y_ = estimate.y_

In [151]:
# comparaison avec différents prédicteurs "stupides"
y1_ = np.array([np.mean(y_)]*len(y_))
print("dummy estimator : mean of |y1_ - y_|   :   "+str(np.mean(np.abs(y1_-y_))))
print("dummy estimator : explained variance : "+str(metrics.explained_variance_score(y_,y1_)))
print("dummy estimator : mean absolute error : "+str(metrics.mean_absolute_error(y_,y1_)))
print("dummy estimator : mean squared error : "+str(metrics.mean_squared_error(y_,y1_)))
print("dummy estimator : r2 score : "+str(metrics.r2_score(y_,y1_)))
    
# random forest
print("randomforest : mean of |ypred_randomforest - y_|   :   "+str(np.mean(np.abs(ypred_randomforest-y_))))
print("randomforest : explained variance : "+str(metrics.explained_variance_score(y_,ypred_randomforest)))
print("randomforest : mean absolute error : "+str(metrics.mean_absolute_error(y_,ypred_randomforest)))
print("randomforest : mean squared error : "+str(metrics.mean_squared_error(y_,ypred_randomforest)))
print("randomforest : r2 score : "+str(metrics.r2_score(y_,ypred_randomforest)))
    
# svm
print("svm : mean of |ypred_svm - y_|   :   "+str(np.mean(np.abs(ypred_svm-y_))))
print("svm : explained variance : "+str(metrics.explained_variance_score(y_,ypred_svm)))
print("svm : mean absolute error : "+str(metrics.mean_absolute_error(y_,ypred_svm)))
print("svm : mean squared error : "+str(metrics.mean_squared_error(y_,ypred_svm)))
print("svm : r2 score : "+str(metrics.r2_score(y_,ypred_svm)))

dummy estimator : mean of |y1_ - y_|   :   0.7549927545245116
dummy estimator : explained variance : 0.0
dummy estimator : mean absolute error : 0.7549927545245116
dummy estimator : mean squared error : 0.9128747333904755
dummy estimator : r2 score : 0.0
randomforest : mean of |ypred_randomforest - y_|   :   0.6874951937186715
randomforest : explained variance : 0.1714866161959434
randomforest : mean absolute error : 0.6874951937186715
randomforest : mean squared error : 0.7563359887278474
randomforest : r2 score : 0.1714788885450178
svm : mean of |ypred_svm - y_|   :   0.6660142966028574
svm : explained variance : 0.21592408396442087
svm : mean absolute error : 0.6660142966028574
svm : mean squared error : 0.7171962853531845
svm : r2 score : 0.21435410673546484
